# Example Notebook for evaluation of FIRESTARTER optimization results

In [ ]:
import json
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

#### Defining the used metrics

In [ ]:
POWER = 'sysfs-powercap-rapl'
IPC = 'perf-ipc'
IPC_2 = 'ipc-estimate'
FREQ = 'perf-freq'
ITEMS = [POWER, IPC, IPC_2, FREQ]

#### Read the json output of FIRESTARTER

In [ ]:
with open('testmachine_DATETIME.json', 'r') as fp:
    data = json.loads(fp.read())
    startTime = data['startTime']
    del data['startTime']
    endTime = data['endTime']
    del data['endTime']
    args = data['args']
    del data['args']
    hostname = data['hostname']
    del data['hostname']
    payloadItems = data['payloadItems']
    del data['payloadItems']
df = pd.DataFrame(data)

print(f"Evaluating FIRESTARTER run on {hostname} from {startTime} to {endTime} with args: {args}")

In [ ]:
for v in ITEMS:
    df[v] = df['metrics'].apply(lambda arr: round(arr[v]['average'], 2))
    
df = df.drop(columns=['metrics'])
df

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18,6))

p = sns.scatterplot(data=df, x=FREQ, y=POWER, hue=range(len(df)), ax=ax1)
p.set_xlim(0)
p.invert_xaxis()

p = sns.scatterplot(data=df, x=POWER, y=IPC, hue=range(len(df)), ax=ax2)
p.set_ylim(0)

p = sns.scatterplot(data=df, x=FREQ, y=IPC, hue=range(len(df)), ax=ax3)
p.set_xlim(0)
p.invert_xaxis()
p.set_ylim(0)

#### This shows the correlation between the perf-ipc and ipc-estimate metric

In [ ]:
p = sns.scatterplot(data=df, x=IPC, y=IPC_2, hue=range(len(df)))

#### Show the evaluated individuals sorted by Power

In [ ]:
df.sort_values(by=POWER, ascending=False)

In [ ]:
def generate_run_instruction_groups_str(items: str, ind: [int]) -> str:
    z = zip(items, ind)
    z = filter(lambda v: v[1] != 0, z)
    z = map(lambda v: v[0] + ":" + str(v[1]), z)
    return ','.join(z)

In [ ]:
best = df.sort_values(by=POWER, ascending=False)['individuals'].tolist()[0]
print(f"""To run FIRESTARTER with the best individual of metric {POWER} use the command line argument
--run-instruction-groups={generate_run_instruction_groups_str(payloadItems, best)}""")